我们考虑一个打车订单分配问题，假设有乘客rider集合R，司机driver的集合D，我们考虑顾客数量和司机数量相等的情况，来将顾客和司机进行一对一的匹配，这个问题可以建模为一个整数规划Integer Programming，但是这个问题非常特殊，将binary constraints松弛掉，依然存在整数最优解。

In [56]:
from gurobipy import *
import pandas as pd 
import numpy as np 
import random

In [57]:
# 生成一个两两收益矩阵20个乘客，20个司机
order_num = 20   
profit_matrix = np.zeros((order_num, order_num))
for i in range(order_num):
    for j in range(order_num):
        random.seed(i * order_num + j) 
        profit_matrix[i][j] = round(10 * random.random(), 1) 

profit_matrix 

array([[8.4, 1.3, 9.6, 2.4, 2.4, 6.2, 7.9, 3.2, 2.3, 4.6, 5.7, 4.5, 4.7,
        2.6, 1.1, 9.7, 3.6, 5.2, 1.8, 6.8],
       [9.1, 1.6, 9.6, 9.2, 7.1, 3.8, 7.5, 6.5, 1.1, 5.5, 5.4, 0.1, 0.8,
        5.7, 5.3, 5.5, 3.3, 6.8, 6.4, 2.1],
       [4.6, 3.8, 6.4, 0.4, 4.1, 2.7, 8.9, 3.5, 5.5, 0.7, 5. , 2.4, 9.8,
        6.2, 9.1, 0.9, 9.7, 0.4, 5.8, 2.2],
       [3.1, 4.9, 9.3, 4.5, 4.8, 4.1, 0.7, 0.7, 7.4, 6.8, 9.1, 3.2, 0.7,
        2.8, 8.6, 4.5, 3.7, 8. , 8.1, 1.5],
       [2.7, 5.1, 1.4, 5. , 7.3, 2. , 7.9, 1.5, 4. , 0.8, 2. , 0.8, 4.2,
        9.1, 5.5, 7.6, 3.7, 1.9, 3.6, 4. ],
       [1.5, 5.8, 1.5, 9.8, 9.6, 8.8, 7.1, 2.5, 1.3, 2.8, 9.3, 8.3, 4.8,
        0.3, 2.4, 9. , 8.2, 2.4, 7.1, 9.3],
       [5.1, 0.9, 5.1, 0.5, 9.6, 9. , 5.7, 0.4, 9.4, 5.9, 5.2, 3.2, 4.1,
        4.9, 4.8, 6.7, 5.8, 0.7, 1.9, 0. ],
       [7.7, 5.2, 5.8, 9.1, 4.5, 8.7, 1.3, 6.1, 4. , 0.7, 8.2, 7.1, 9.8,
        9.9, 2. , 5.8, 4.9, 6.1, 6.8, 3.9],
       [1.2, 5.2, 0.5, 5.4, 1. , 0.2, 1.8, 2.3, 7.5, 2.6, 7.9, 7

In [58]:
P = pd.DataFrame(profit_matrix)
P 

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,8.4,1.3,9.6,2.4,2.4,6.2,7.9,3.2,2.3,4.6,5.7,4.5,4.7,2.6,1.1,9.7,3.6,5.2,1.8,6.8
1,9.1,1.6,9.6,9.2,7.1,3.8,7.5,6.5,1.1,5.5,5.4,0.1,0.8,5.7,5.3,5.5,3.3,6.8,6.4,2.1
2,4.6,3.8,6.4,0.4,4.1,2.7,8.9,3.5,5.5,0.7,5.0,2.4,9.8,6.2,9.1,0.9,9.7,0.4,5.8,2.2
3,3.1,4.9,9.3,4.5,4.8,4.1,0.7,0.7,7.4,6.8,9.1,3.2,0.7,2.8,8.6,4.5,3.7,8.0,8.1,1.5
4,2.7,5.1,1.4,5.0,7.3,2.0,7.9,1.5,4.0,0.8,2.0,0.8,4.2,9.1,5.5,7.6,3.7,1.9,3.6,4.0
5,1.5,5.8,1.5,9.8,9.6,8.8,7.1,2.5,1.3,2.8,9.3,8.3,4.8,0.3,2.4,9.0,8.2,2.4,7.1,9.3
6,5.1,0.9,5.1,0.5,9.6,9.0,5.7,0.4,9.4,5.9,5.2,3.2,4.1,4.9,4.8,6.7,5.8,0.7,1.9,0.0
7,7.7,5.2,5.8,9.1,4.5,8.7,1.3,6.1,4.0,0.7,8.2,7.1,9.8,9.9,2.0,5.8,4.9,6.1,6.8,3.9
8,1.2,5.2,0.5,5.4,1.0,0.2,1.8,2.3,7.5,2.6,7.9,7.8,3.5,3.2,8.0,4.4,0.3,9.7,9.5,9.0
9,1.4,5.8,4.5,1.1,2.8,9.0,3.4,3.5,7.9,4.8,0.7,7.6,3.7,5.6,8.1,6.0,6.5,5.4,0.5,3.4


In [59]:
model = Model('Assignment_Problem') 

In [60]:
# 创建变量
x = [[[] for i in range(order_num)] for j in range(order_num)]
x

[[[],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  []],
 [[],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  []],
 [[],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  []],
 [[],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  []],
 [[],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  []],
 [[],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  []],
 [[],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  []],
 [[],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  []],
 [[],
  [],
  [],
  [],
  [],
  

In [61]:
for i in range(order_num):
    for j in range(order_num): 
        x[i][j] = model.addVar(lb = 0
                               ,ub = 1 
                               ,vtype = GRB.CONTINUOUS   # decision variable type
                               ,name = "x_" + str(i) + "_" + str(j)  
                               )
x

[[<gurobi.Var *Awaiting Model Update*>,
  <gurobi.Var *Awaiting Model Update*>,
  <gurobi.Var *Awaiting Model Update*>,
  <gurobi.Var *Awaiting Model Update*>,
  <gurobi.Var *Awaiting Model Update*>,
  <gurobi.Var *Awaiting Model Update*>,
  <gurobi.Var *Awaiting Model Update*>,
  <gurobi.Var *Awaiting Model Update*>,
  <gurobi.Var *Awaiting Model Update*>,
  <gurobi.Var *Awaiting Model Update*>,
  <gurobi.Var *Awaiting Model Update*>,
  <gurobi.Var *Awaiting Model Update*>,
  <gurobi.Var *Awaiting Model Update*>,
  <gurobi.Var *Awaiting Model Update*>,
  <gurobi.Var *Awaiting Model Update*>,
  <gurobi.Var *Awaiting Model Update*>,
  <gurobi.Var *Awaiting Model Update*>,
  <gurobi.Var *Awaiting Model Update*>,
  <gurobi.Var *Awaiting Model Update*>,
  <gurobi.Var *Awaiting Model Update*>],
 [<gurobi.Var *Awaiting Model Update*>,
  <gurobi.Var *Awaiting Model Update*>,
  <gurobi.Var *Awaiting Model Update*>,
  <gurobi.Var *Awaiting Model Update*>,
  <gurobi.Var *Awaiting Model Update*>,

In [62]:
# 更新变量环境
model.update()

In [63]:
# 目标函数
obj = LinExpr(0)
obj

<gurobi.LinExpr: 0.0>

In [64]:
for i in range(order_num):
    for j in range(order_num): 
        obj.addTerms(profit_matrix[i][j], x[i][j])
obj

<gurobi.LinExpr: 8.4 x_0_0 + 1.3 x_0_1 + 9.6 x_0_2 + 2.4 x_0_3 + 2.4 x_0_4 + 6.2 x_0_5 + 7.9 x_0_6 + 3.2 x_0_7 + 2.3 x_0_8 + 4.6 x_0_9 + 5.7 x_0_10 + 4.5 x_0_11 + 4.7 x_0_12 + 2.6 x_0_13 + 1.1 x_0_14 + 9.7 x_0_15 + 3.6 x_0_16 + 5.2 x_0_17 + 1.8 x_0_18 + 6.8 x_0_19 + 9.1 x_1_0 + 1.6 x_1_1 + 9.6 x_1_2 + 9.2 x_1_3 + 7.1 x_1_4 + 3.8 x_1_5 + 7.5 x_1_6 + 6.5 x_1_7 + 1.1 x_1_8 + 5.5 x_1_9 + 5.4 x_1_10 + 0.1 x_1_11 + 0.8 x_1_12 + 5.7 x_1_13 + 5.3 x_1_14 + 5.5 x_1_15 + 3.3 x_1_16 + 6.8 x_1_17 + 6.4 x_1_18 + 2.1 x_1_19 + 4.6 x_2_0 + 3.8 x_2_1 + 6.4 x_2_2 + 0.4 x_2_3 + 4.1 x_2_4 + 2.7 x_2_5 + 8.9 x_2_6 + 3.5 x_2_7 + 5.5 x_2_8 + 0.7 x_2_9 + 5.0 x_2_10 + 2.4 x_2_11 + 9.8 x_2_12 + 6.2 x_2_13 + 9.1 x_2_14 + 0.9 x_2_15 + 9.7 x_2_16 + 0.4 x_2_17 + 5.8 x_2_18 + 2.2 x_2_19 + 3.1 x_3_0 + 4.9 x_3_1 + 9.3 x_3_2 + 4.5 x_3_3 + 4.8 x_3_4 + 4.1 x_3_5 + 0.7 x_3_6 + 0.7 x_3_7 + 7.4 x_3_8 + 6.8 x_3_9 + 9.1 x_3_10 + 3.2 x_3_11 + 0.7 x_3_12 + 2.8 x_3_13 + 8.6 x_3_14 + 4.5 x_3_15 + 3.7 x_3_16 + 8.0 x_3_17 + 8.1 x_3_1

In [65]:
model.setObjective(obj, GRB.MAXIMIZE)

In [66]:
# 创建约束条件1
for j in range(order_num):
    expr = LinExpr(0)
    for i in range(order_num):  
        expr.addTerms(1, x[i][j]) 
    model.addConstr(expr == 1, name="D_" + str(i)) 

In [67]:
# 创建约束条件2
for i in range(order_num):
    expr = LinExpr(0)
    for j in range(order_num):  
        expr.addTerms(1, x[i][j]) 
    model.addConstr(expr == 1, name="R_" + str(i)) 

In [68]:
# 求解
model.optimize()

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 40 rows, 400 columns and 800 nonzeros
Model fingerprint: 0xa4f67fe0
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-01, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.00s
Presolved: 40 rows, 400 columns, 800 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.8840000e+02   1.500000e+01   0.000000e+00      0s
      11    1.8650000e+02   0.000000e+00   0.000000e+00      0s

Solved in 11 iterations and 0.01 seconds
Optimal objective  1.865000000e+02


In [69]:
for var in model.getVars():
    if(var.x > 0):
        print(var.varName, '\t', var.x) 

x_0_15 	 1.0
x_1_0 	 1.0
x_2_16 	 1.0
x_3_10 	 1.0
x_4_13 	 1.0
x_5_19 	 1.0
x_6_4 	 1.0
x_7_12 	 1.0
x_8_18 	 1.0
x_9_5 	 1.0
x_10_17 	 1.0
x_11_8 	 1.0
x_12_3 	 1.0
x_13_6 	 1.0
x_14_9 	 1.0
x_15_2 	 1.0
x_16_7 	 1.0
x_17_11 	 1.0
x_18_1 	 1.0
x_19_14 	 1.0
